# Quantum Circuit Simulator

Sophie Cavallini | 2021

First of all I need to import the necesserary libraries. Since the idea of the task is to create a quantum simulator from scratch I won't add any library with build-in function for quantum computing like qiskit and cirq.

In [1]:
import numpy as np
import math
import cmath
import numpy.random

## Part 1
In this part I created some basic matrixes/vector that are usefull, so I don't have to define them everytime I need use them.
### Basis:
Z basis (|0> and |1>), X basis (|+> and |->) and Y basis (|i> and |-i>).

Note: I use the encoding Big Endian.

In [2]:
# Qubit in |0> state (100% probability of measuring 0)
q0 = [1, 0]

# Qubit in |1> state (100% probability of measuring 1)
q1 = [0, 1] 

# Qubit |+> state (superposition: 50% probability of measuring 0 and 50% probability of measuring 1)
q2 = [0.7071067811865475, 0.7071067811865475]

# Qubit |-> state (superposition: 50% probability of measuring 0 and 50% probability of measuring 1) with phase pi
q3 = [0.7071067811865475, -0.7071067811865475]

# Qubit |i> state (superposition: 50% probability of measuring 0 and 50% probability of measuring 1) with phase pi/2
q3 = [0.7071067811865475, 0+0.7071067811865475j]

# Qubit |-i> state (superposition: 50% probability of measuring 0 and 50% probability of measuring 1) with phase -pi/2
q4 = [0.7071067811865475, 0-0.7071067811865475j]

### Gates:
X, Z, H(Hadamard), Y, S, T, CNOT and SWAP.


In [ ]:
#Define X (NOT) gate:
X = np.array([[0, 1], [1, 0]])

#Define Z gate
Z = np.array([[1, 0], [0, -1]])

# Define H (Hadamard) gate:
H = np.array([[1/np.sqrt(2), 1/np.sqrt(2)], [1/np.sqrt(2), -1/np.sqrt(2)]])

#Define Y gate:
Y = np.array([[0, -1j], [1j, 0]])

#Define S gate
S = np.array([[1, 0], [0, cmath.exp(1j*math.pi/2)]])

#Define T gate
T = np.array([[1, 0],[0, cmath.exp(1j*math.pi/4)]])

#Define CNOT gate
CX = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]])

#Define SWAP gate
SWAP = [[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0, 0, 0, 1]]

### Other usefull things to have:

In [ ]:
# Define projection operator |0><0|
P0x0 = np.array([[1, 0],[0, 0]])

# Define projection operator |1><1|
P1x1 = np.array([[0, 0],[0, 1]])

#Define identity matrix
I = np.identity(2)

## Part 2
Here I created all the simulator itself, so I made the functions necessary to:
- initialize state
- read program, and for each gate:
  - calculate matrix operator
  - apply operator (modify state)
- perform multi-shot measurement of all qubits using weighted random technique.

This is the complete simulator [qc_basic.py](https://github.com/SophieCavallini/QOSF-task/blob/main/qc_basic.py).

### Initialize state:

In [ ]:
def get_ground_state(num_qubits):
    #return vector of size 2**num_qubits with all zeroes except first element which is 1
    vq = np.zeros_like(np.arange(2**num_qubits))
    vq[0] = 1
    return vq

### Calculate the matrix operator:
If the gate is unitary it simply check between the gates that I created above and outputs the correct unitary given the name. This function only apply if the gate a single qubit gate. It checks it in the run_program function.

In [ ]:
def find_unitary(name_gate):
    # return the unitary matrix of single-qubit elementary gate, given the gate name
    if(name_gate == 'x'): return X
    if(name_gate == 'z'): return Z
    if(name_gate == 'h'): return H
    if(name_gate == 'y'): return Y
    if(name_gate == 's'): return S
    if(name_gate == 't'): return T
    else:
        print('The gate: ', name_gate, 'does not exists, yet, the program will go to the next one' )
        return I

Then it has get the operator given the matrix unitary of the gate. It has to apply the gate in the correct position given the target qubit. Also this function applies only to single-qubit gates.

In [ ]:
def get_operator(total_qubits, gate_unitary, target_qubits):
    # return unitary operator of size 2**n x 2**n for given single-qubit gate and target qubits
    I = np.identity(2)
    Ik = I
    if target_qubits==0:
        O=gate_unitary
    else:
        for i in range(1, target_qubits-1):
            Ik = np.kron(Ik, I)
        O = np.kron(Ik, gate_unitary) 
    for i in range(1, total_qubits-target_qubits):
        O = np.kron(O, I)
    return O

For the CNOT gate the things gate I little bit harder, since it is a multi-qubit gates, it has to handle with target qubit and also the controll qubit. So, I have decided to create a single function which gets the unitary and the applies it in the correct position. This function applies only to the CNOT gate, and not the every 2-qubit gate.

In [ ]:
def cnot(control, target, num_qubits):
    # return unitary operator of size 2**n x 2**n for cnot gate given the control and target qubits
    if(control>target):
        #if the control qubit is larger than the target one it will compute the unitary operator with target and control inverted and then
        #rotate and traspose the matrix to obtain the one required
        return np.rot90(cnot(target, control, num_qubits),2).T
    #P0x0 and P1x1 need to be in position: control (so if control = 0 then they will be the first factor of the tensor product)
    if (control == 0):
        O1 = P0x0
        O2 = P1x1
    else:
        O1 = I
        for i in range(1, control):
            O1 = np.kron(O1, I)
        O2 = np.kron(O1, P1x1)
        O1 = np.kron(O1, P0x0)
    #in position: target, for O2 there will be the X unitary, will for O1 there will be simply the identity matrix
    for i in range(1, np.abs(target-control)):
        O1 = np.kron(O1, I)
        O2 = np.kron(O2, I)
    O1 = np.kron(O1, I)
    O2 = np.kron(O2, X)
    for i in range(1, num_qubits-target):
        O1 = np.kron(O1, I)
        O2 = np.kron(O2, I)
    O = O1 + O2
    return O

### Read program:

In [ ]:
def run_program(initial_state, program):
    size = len(initial_state)
    num_qubits = int(math.log2(size))
    final_state = initial_state
    for i in range(0, len(program)):
        if(len((program[i])["target"]) == 1):
            target = ((program[i])["target"])[0]

            unitary = find_unitary((program[i])["gate"])
            operator = get_operator(num_qubits, unitary, target)
        elif((program[i])["gate"] == 'cx'):
            #if the gate is cnot it will compute the operator via the specific cnot function (it will not use get_operator)
            operator = cnot(((program[i])["target"])[0], ((program[i])["target"])[1], num_qubits)
        else:
            print('The gate: ', (program[i])["gate"], 'does not exists, yet, the program will go to the next one' )
        final_state = np.dot(final_state, operator)
    
    return final_state

### Measurment:

In [ ]:
def measure_all(state_vector):
    # choose element from state_vector using weighted random and return it's index
    pstate_vector = np.abs(state_vector)**2
    a = numpy.random.rand(1)[0]
    i = 0
    prec = 0
    while a > pstate_vector[i]+prec:
        prec = prec + pstate_vector[i]
        i = i+1
    return i

In [ ]:
def get_counts(state_vector, num_shots):
    # simply execute measure_all in a loop num_shots times and return object with statistics
    occurences = np.zeros_like(np.arange(len(state_vector)))
    for i in range(0, num_shots):
        place = measure_all(state_vector)
        occurences[place] = occurences[place]+1
    # convert the array in a dictionary, so it can be easier to understand the output    
    odict = {}
    type(odict)
    for i in range(0, len(occurences)):
        if occurences[i]>0: odict["{0:b}".format(i)] = occurences[i]
    return odict

You can see an example of usage of the simulator below:

In [ ]:
# Define program:

my_circuit = [
{ "gate": "h", "target": [0] }, 
{ "gate": "cx", "target": [0, 1] }
]

# Create "quantum computer" with 2 qubits (this is actually just a vector :) )

my_qpu = get_ground_state(2)

# Run circuit

final_state = run_program(my_qpu, my_circuit)

# Read results

counts = get_counts(final_state, 1000)

print(counts)

## Part 3
In this part I've implemented additional requirments of the task.

## Parametric gates:
In the case of parametric gates (single-qubit gate) it has evaluate the unitary given the parameter.

Complete program: [qc_parametric_gates.py](https://github.com/SophieCavallini/QOSF-task/blob/main/qc_parametric_gates.py).

In [ ]:
def u3_unitary(param):
    # return the unitary of a parametric gate given the parameters
    theta = param["theta"]
    phi = param["phi"]
    lam = param["lambda"]
    U = [[math.cos(theta/2), -cmath.exp(1j * lam) * math.sin(theta / 2)], 
         [cmath.exp(1j * phi) * math.sin(theta / 2), cmath.exp(1j * lam + 1j * phi) * math.cos(theta / 2)]]
    return U

To enable the usage of parametric gates, I also have to change a bit, the run_program function.

In [ ]:
def run_program(initial_state, program):
    size = len(initial_state)
    num_qubits = int(math.log2(size))
    final_state = initial_state
    for i in range(0, len(program)):
        if(len((program[i])["target"]) == 1):
            target = ((program[i])["target"])[0]
            if((program[i])["gate"]=="u3"):
                #for parametric gates
                unitary = u3_unitary((program[i])["params"])
            else:
                #for elementary gates
                unitary = find_unitary((program[i])["gate"])
            operator = get_operator(num_qubits, unitary, target)
        elif((program[i])["gate"] == 'cx'):
            #if the gate is cnot it will compute the operator via the specific cnot function (it will not use get_operator)
            operator = cnot(((program[i])["target"])[0], ((program[i])["target"])[1], num_qubits)
        else:
            print('The gate: ', (program[i])["gate"], 'does not exists, yet, the program will go to the next one' )
        final_state = np.dot(final_state, operator)
    
    return final_state

You should give the parametric gates to the simulator as in the following example:

In [ ]:
[
  { "unitary": [["cos(theta/2)", "-exp(i * lambda) * sin(theta / 2)"], ["exp(i * phi) * sin(theta / 2)", "exp(i * lambda + i * phi) * cos(theta / 2)"]], "params": { "theta": 3.1415, "phi": 1.15708, "lambda": -3.1415 }, "target": [0] }
  ...
]

### Running Variational Quantum Algorithms
With support for parametric gates, all you need to do is to allow global params - and your simulator will be able to run variational quantum algorithms!
To be able to run variational quantum algorithms all is needed, with the support for parametric gates, is to allow global parameters.
So I had to change a bit the u3_unitary function created above.

In [ ]:
def u3_unitary(param, gparam):
    # return the unitary of a parametric gate given the parameters
    
    #if there are global parameters it will used them, else it will you use the parameter given in the circuit
    if(param["theta"]=="global_1"): theta = gparam["global_1"]
    else: theta = param["theta"]
    if(param["phi"]=="global_2"): phi = param["global_2"]
    else: phi = param["phi"]
    lam = param["lambda"]
    U = [[math.cos(theta/2), -cmath.exp(1j * lam) * math.sin(theta / 2)], 
         [cmath.exp(1j * phi) * math.sin(theta / 2), cmath.exp(1j * lam + 1j * phi) * math.cos(theta / 2)]]
    return U


Also in this case I had to make some minors changing to run_program.

In [ ]:
def run_program(initial_state, program, gparams):
    size = len(initial_state)
    num_qubits = int(math.log2(size))
    final_state = initial_state
    for i in range(0, len(program)):
        if(len((program[i])["target"]) == 1):
            target = ((program[i])["target"])[0]
            if((program[i])["gate"]=="u3"):
                #for parametric gates
                unitary = u3_unitary((program[i])["params"], gparams)
            else:
                #for elementary gates
                unitary = find_unitary((program[i])["gate"])
            operator = get_operator(num_qubits, unitary, target)
        elif((program[i])["gate"] == 'cx'):
            #if the gate is cnot it will compute the operator via the specific cnot function (it will not use get_operator)
            operator = cnot(((program[i])["target"])[0], ((program[i])["target"])[1], num_qubits)
        else:
            print('The gate: ', (program[i])["gate"], 'does not exists, yet, the program will go to the next one' )
        final_state = np.dot(final_state, operator)
    
    return final_state

An example of circuit with global parameters is the following:

In [ ]:
[
  { "gate": "u3", "params": { "theta": "global_1", "phi": "global_2", "lambda": -3.1415 }, "target": [0] }
  ...
]

Add you can pass the global params to run_program:

In [ ]:
final_state = run_program(my_qpu, my_circuit, { "global_1": 3.1415, "global_2": 1.5708 })